In [0]:
t = 4 + 4

In [0]:
# Import functions
from pyspark.sql.functions import col, current_timestamp

# Define variables used in code below
file_path = "/FileStore/stream/events/"
username = spark.sql("SELECT regexp_replace(current_user(), '[^a-zA-Z0-9]', '_')").first()[0]
table_name = f"{username}_etl_quickstart"
checkpoint_path = f"/tmp/{username}/_checkpoint/etl_quickstart"

# Clear out data from previous demo execution
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
dbutils.fs.rm(checkpoint_path, True)



In [0]:
dbutils.fs.rm(f"dbfs:/user/hive/warehouse/{username}_etl_quickstart", True)


In [0]:
%fs ls /databricks-datasets/structured-streaming/events

Żeby sprawdzić czy stream działa kopjuj po jednym bądź kilku plikach 

In [0]:
dbutils.fs.cp("/databricks-datasets/structured-streaming/events/file-4.json","/FileStore/stream/events/file-4.json",True)

In [0]:
display(dbutils.fs.ls("/FileStore/stream/events/"))

In [0]:
spark.read.format("json").load("dbfs:/FileStore/stream/events/").count()

Dokończyć kod autoloadera 
1. Dodaj opcje 'cloudfiles'
2. Dodaj kolumnę z metadanych 'source_file'
3. Dane zapisać do tabeli

In [0]:
%python
from pyspark.sql.functions import input_file_name

(spark.readStream
  .format("cloudFiles")  
  .option("cloudFiles.format", "json")  
  .load(file_path) 

  #  Dodaj kolumnę z metadanych
  .withColumn("source_file", input_file_name())

  # Zapisz dane do tabeli
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  .toTable(table_name)  # zapis jako tabela zarządzana przez Spark
)

In [0]:
df = spark.sql(f"select count(*) from {table_name}")
df.display()

Sprawdzć metadane 

In [0]:
display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))

In [0]:
%fs ls /tmp/{user}/_checkpoint/etl_quickstart/metadata